In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving movies.dat to movies.dat
User uploaded file "movies.dat" with length 171308 bytes


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving users.dat to users.dat
User uploaded file "users.dat" with length 134368 bytes


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving ratings.dat to ratings.dat
User uploaded file "ratings.dat" with length 24594131 bytes


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving u1.base to u1.base
User uploaded file "u1.base" with length 1586544 bytes


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
!pip install torch===1.4.0 torchvision===0.5.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
!update pytouch

/bin/bash: update: command not found


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importing the dataset
movies = pd.read_csv('movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

# Preparing the training set and the test set
training_set = pd.read_csv('u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')


In [ ]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [ ]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)


In [ ]:
nb_users

943

In [ ]:
nb_movies

1682

In [ ]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [ ]:
# Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

# Creating the architecture of the Neural Network

class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)   #100*1682                 
        self.a = torch.randn(1, nh)    #1*100                       
        self.b = torch.randn(1, nv)    #1*1682                   
    def sample_h(self, x):                                     
        wx = torch.mm(x, self.W.t())    #100*1682 * 1682*100 = 100*100
        activation = wx + self.a.expand_as(wx)                  
        p_h_given_v = torch.sigmoid(activation)     #100*100
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):                                      
        wy = torch.mm(y, self.W)    #100*100 * 100*1682 = 100*1682
        activation = wy + self.b.expand_as(wy)                  
        p_v_given_h = torch.sigmoid(activation)     #100*1682
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):                         
        self.W += (torch.mm(v0.t(), ph0)- torch.mm(vk.t(), phk)).t()    #100*1682 
                             #!= 1682*100 * 100*100 - 1682*100 * 100*100 = 1682*100
        self.b += torch.sum((v0-vk), 0)                            
        self.a += torch.sum((ph0-phk), 0)  

In [ ]:
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)  
print(rbm.W.size())

torch.Size([100, 1682])


In [ ]:
# Training the RBM
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user + batch_size]
        v0 = training_set[id_user:id_user + batch_size]
        ph0, _ = rbm.sample_h(v0)
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            vk[v0 < 0] = v0[v0 < 0]
        phk, _ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        s += 1.
    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss / s))


epoch: 1 loss: tensor(0.3476)
epoch: 2 loss: tensor(0.2449)
epoch: 3 loss: tensor(0.2507)
epoch: 4 loss: tensor(0.2500)
epoch: 5 loss: tensor(0.2462)
epoch: 6 loss: tensor(0.2441)
epoch: 7 loss: tensor(0.2490)
epoch: 8 loss: tensor(0.2472)
epoch: 9 loss: tensor(0.2483)
epoch: 10 loss: tensor(0.2463)


In [ ]:
# Testing the RBM
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2474)


In [ ]:
#########################